In [3]:
import pandas as pd
import numpy as np
import os
from tensorflow.keras.utils import to_categorical

# 일상 데이터[0,1,0](주로 배회)

#### 데이터 모두 하이라이트에 근접했다고 판단하여 필터링 skip

#### average pooling 사용

In [9]:
# 디렉토리 경로 설정
directory_path = './sequential_datas/walking' 

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        filepath = os.path.join(directory_path, filename)

        # csv 파일 로드
        df = pd.read_csv(filepath)

        # frame 번호에 대한 열 추가하기 (0부터 시작)
        df['frame_number'] = np.arange(len(df))

        # frame 번호를 3으로 나누어 정수 부분만 취하면, 연속된 세 프레임이 같은 그룹으로 묶일 것입니다.
        df['group'] = df['frame_number'] // 3

        # 각 그룹에 대해 평균값 계산하기
        df_averaged = df.groupby('group').mean()

        # 결과 저장하기 
        output_filepath = filepath.replace('.csv', '_averaged.csv')
        df_averaged.to_csv(output_filepath, index=False)

In [10]:
# 4가지 관절 각도에 대한 열 이름
angle_columns = ['elbow', 'shoulder', 'hip', 'knee']

# 디렉토리 경로 설정
directory_path = './sequential_datas/walking' 

X_train_list = []
y_train_list = []

for filename in os.listdir(directory_path):
    if filename.endswith('_averaged.csv'):
        filepath = os.path.join(directory_path, filename)

        # csv 파일 로드
        df = pd.read_csv(filepath)

        # 관절 각도 데이터 가져오기
        data = df[angle_columns].values

        # 5프레임 동안의 관절각도 변화를 포함하는 리스트 생성하기
        sequences = [data[i:i+5] for i in range(len(data)-4)]

        X_train_list.extend(sequences)
        
        # 타겟 설정하기 
        y_train_list.extend([[0,1,0]] * len(sequences))

# 배열 형태로 변환하기
X_train = np.array(X_train_list)
y_train = np.array(y_train_list)

print(f"Training data shape: {X_train.shape}")  # 출력 결과: (전체 데이터 개수, 시퀀스 길이, feature 개수)
print(f"Target shape: {y_train.shape}")   # 출력 결과: (전체 데이터 개수, 클래스 개수)

Training data shape: (1164, 5, 4)
Target shape: (1164, 3)


In [12]:
y_train[0]

array([0, 1, 0])

# 낙상 데이터[1,0,0]

## 변화가 미비한 구간에 대한 필터링

In [27]:
# 4가지 관절 각도에 대한 열 이름
angle_columns = ['elbow', 'shoulder', 'hip', 'knee']

# rolling window 크기 설정
window_size = 5

# 임계값 설정
threshold = 0.5  

# 보존할 주변 프레임 수 설정
buffer_frames = 5   

directory_path = './sequential_datas/falling'

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        filepath = os.path.join(directory_path, filename)

        # csv 파일 로드
        df = pd.read_csv(filepath)

        for col in angle_columns:
            # rolling window 적용하여 표준 편차 계산하기
            df[f'{col}_std'] = df[col].rolling(window_size).std()

        movement_flags = (df[[f'{col}_std' for col in angle_columns]] < threshold).all(axis=1)

        for i in range(-buffer_frames, buffer_frames + 1):
            movement_flags |= movement_flags.shift(i)

        df_movement_preserved = df[~movement_flags]

        # 결과 저장하기 
        output_filepath = filepath.replace('.csv', '_movement_preserved.csv')
        df_movement_preserved.to_csv(output_filepath, index=False)

        print(f"For {filename}, number of rows changed from {len(df)} to {len(df_movement_preserved)}")


For falling (1).mp4_angles.csv, number of rows changed from 132 to 60
For falling (10).mp4_angles.csv, number of rows changed from 104 to 77
For falling (11).mp4_angles.csv, number of rows changed from 76 to 49
For falling (12).mp4_angles.csv, number of rows changed from 77 to 77
For falling (13).mp4_angles.csv, number of rows changed from 65 to 65
For falling (14).mp4_angles.csv, number of rows changed from 85 to 52
For falling (15).mp4_angles.csv, number of rows changed from 97 to 97
For falling (16).mp4_angles.csv, number of rows changed from 114 to 45
For falling (17).mp4_angles.csv, number of rows changed from 84 to 84
For falling (18).mp4_angles.csv, number of rows changed from 75 to 75
For falling (19).mp4_angles.csv, number of rows changed from 130 to 85
For falling (2).mp4_angles.csv, number of rows changed from 103 to 103
For falling (20).mp4_angles.csv, number of rows changed from 98 to 98
For falling (21).mp4_angles.csv, number of rows changed from 133 to 106
For falling (2

In [ ]:
# 디렉토리 경로 설정
directory_path = './sequential_datas/falling' 

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        filepath = os.path.join(directory_path, filename)

        # csv 파일 로드
        df = pd.read_csv(filepath)

        # frame 번호에 대한 열 추가하기 (0부터 시작)
        df['frame_number'] = np.arange(len(df))

        # frame 번호를 3으로 나누어 정수 부분만 취하면, 연속된 세 프레임이 같은 그룹으로 묶일 것입니다.
        df['group'] = df['frame_number'] // 3

        # 각 그룹에 대해 평균값 계산하기
        df_averaged = df.groupby('group').mean()

        # 결과 저장하기 
        output_filepath = filepath.replace('.csv', '_averaged.csv')
        df_averaged.to_csv(output_filepath, index=False)

In [14]:
# 낙상 데이터가 있는 디렉토리 경로 설정
fall_directory_path = './sequential_datas/falling' 

X_fall_list = []
y_fall_list = []

for filename in os.listdir(fall_directory_path):
    if filename.endswith('_averaged.csv'):
        filepath = os.path.join(fall_directory_path, filename)

        # csv 파일 로드
        df = pd.read_csv(filepath)

        # 관절 각도 데이터 가져오기
        data = df[angle_columns].values

        # 5프레임 동안의 관절각도 변화를 포함하는 리스트 생성하기
        sequences = [data[i:i+5] for i in range(len(data)-4)]

        X_fall_list.extend(sequences)
        
        # 타겟 설정하기 ([1,0,0]으로 설정)
        y_fall_list.extend([[1,0,0]] * len(sequences))

# 배열 형태로 변환하기
X_fall_train = np.array(X_fall_list)
y_fall_train = np.array(y_fall_list)

print(f"Fall Training data shape: {X_fall_train.shape}")  # 출력 결과: (낙상 데이터 개수, 시퀀스 길이, feature 개수)
print(f"Fall Target shape: {y_fall_train.shape}")   # 출력 결과: (낙상 데이터 개수, 클래스 개수)

Fall Training data shape: (736, 5, 4)
Fall Target shape: (736, 3)


### 기존 train data와 같은 개수만큼 합치기

In [15]:
# 일상데이터에서 무작위로 fall_data_count 개수만큼 샘플 선택하기
indices = np.random.choice(X_train.shape[0], X_fall_train.shape[0], replace=False)
X_daily_sampled = X_train[indices]
y_daily_sampled = y_train[indices]

print(f"Sampled daily Training data shape: {X_daily_sampled.shape}")  # 출력 결과: (샘플링된 일상데이터의 수량 , 시퀀스 길이 , feature개수 )
print(f"Sampled daily Target shape: {y_daily_sampled.shape}")   # 출력 결과 : (샘플링된 일상데이터의 수량 , 클래스개수 

Sampled daily Training data shape: (736, 5, 4)
Sampled daily Target shape: (736, 3)


In [16]:
# 기존의 훈련 데이터와 낙상 훈련데이터를 합치기 
X_combined = np.concatenate((X_daily_sampled, X_fall_train), axis=0)
y_combined = np.concatenate((y_daily_sampled, y_fall_train), axis=0)

print("Combined Training data shape:", X_combined.shape)  
print("Combined Target shape:", y_combined.shape)


Combined Training data shape: (1472, 5, 4)
Combined Target shape: (1472, 3)


# 눕는 데이터[0,0,1](기존의 배회를 대체)

## 변화가 미비한 구간에 대한 필터링
### 눕는 과정과 누운 후는 구분
#### (누운 후나 낙상 후나 모두 구간별 표준편차가 0에 근접할 것으로 추정하므로 다른 구별에 비해 의미가 크게 없다고 판단)

In [28]:
# 4가지 관절 각도에 대한 열 이름
angle_columns = ['elbow', 'shoulder', 'hip', 'knee']

# rolling window 크기 설정
window_size = 5

# 임계값 설정
threshold = 0.5  

# 보존할 주변 프레임 수 설정
buffer_frames = 5   

directory_path = './sequential_datas/lying'

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        filepath = os.path.join(directory_path, filename)

        # csv 파일 로드
        df = pd.read_csv(filepath)

        for col in angle_columns:
            # rolling window 적용하여 표준 편차 계산하기
            df[f'{col}_std'] = df[col].rolling(window_size).std()

        movement_flags = (df[[f'{col}_std' for col in angle_columns]] < threshold).all(axis=1)

        for i in range(-buffer_frames, buffer_frames + 1):
            movement_flags |= movement_flags.shift(i)

        df_movement_preserved = df[~movement_flags]

        # 결과 저장하기 
        output_filepath = filepath.replace('.csv', '_movement_preserved.csv')
        df_movement_preserved.to_csv(output_filepath, index=False)

        print(f"For {filename}, number of rows changed from {len(df)} to {len(df_movement_preserved)}")


For lying (1).mp4_angles.csv, number of rows changed from 134 to 79
For lying (10).mp4_angles.csv, number of rows changed from 125 to 125
For lying (11).mp4_angles.csv, number of rows changed from 135 to 135
For lying (12).mp4_angles.csv, number of rows changed from 108 to 79
For lying (13).mp4_angles.csv, number of rows changed from 184 to 143
For lying (14).mp4_angles.csv, number of rows changed from 207 to 207
For lying (15).mp4_angles.csv, number of rows changed from 197 to 121
For lying (16).mp4_angles.csv, number of rows changed from 192 to 192
For lying (17).mp4_angles.csv, number of rows changed from 190 to 158
For lying (18).mp4_angles.csv, number of rows changed from 157 to 157
For lying (19).mp4_angles.csv, number of rows changed from 166 to 143
For lying (2).mp4_angles.csv, number of rows changed from 120 to 120
For lying (20).mp4_angles.csv, number of rows changed from 164 to 164
For lying (21).mp4_angles.csv, number of rows changed from 174 to 174
For lying (22).mp4_angle

In [17]:
# 디렉토리 경로 설정
directory_path = './sequential_datas/lying' 

for filename in os.listdir(directory_path):
    if filename.endswith('.csv'):
        filepath = os.path.join(directory_path, filename)

        # csv 파일 로드
        df = pd.read_csv(filepath)

        # frame 번호에 대한 열 추가하기 (0부터 시작)
        df['frame_number'] = np.arange(len(df))

        # frame 번호를 3으로 나누어 정수 부분만 취하면, 연속된 세 프레임이 같은 그룹으로 묶일 것입니다.
        df['group'] = df['frame_number'] // 3

        # 각 그룹에 대해 평균값 계산하기
        df_averaged = df.groupby('group').mean()

        # 결과 저장하기 
        output_filepath = filepath.replace('.csv', '_averaged.csv')
        df_averaged.to_csv(output_filepath, index=False)

In [18]:
# 눕기 데이터가 있는 디렉토리 경로 설정
lying_directory_path = './sequential_datas/lying' 

X_lying_list = []
y_lying_list = []

for filename in os.listdir(lying_directory_path):
    if filename.endswith('_averaged.csv'):
        filepath = os.path.join(lying_directory_path, filename)

        # csv 파일 로드
        df = pd.read_csv(filepath)

        # 관절 각도 데이터 가져오기
        data = df[angle_columns].values

        # 5프레임 동안의 관절각도 변화를 포함하는 리스트 생성하기
        sequences = [data[i:i+5] for i in range(len(data)-4)]

        X_lying_list.extend(sequences)
        
        # 타겟 설정하기 ([0,0,1]으로 설정)
        y_lying_list.extend([[0,0,1]] * len(sequences))

# 배열 형태로 변환하기
X_lying_train = np.array(X_lying_list)
y_lying_train = np.array(y_lying_list)

print(f"Lying Training data shape: {X_lying_train.shape}")  # 출력 결과: (눕기 데이터 개수, 시퀀스 길이 , feature 개수)
print(f"Lying Target shape: {y_lying_train.shape}")   # 출력 결과 : (눕기 데이터 개수 , 클래스 개수 )

Lying Training data shape: (2459, 5, 4)
Lying Target shape: (2459, 3)


In [20]:
# 기존의 훈련 데이터에서 무작위로 X_fall_train.shape[0] 만큼 샘플 선택하기 
indices2= np.random.choice(X_lying_train.shape[0], X_fall_train.shape[0], replace=False) 
X_lying_sampled= X_lying_train[indices2]
y_lying_sampled= y_lying_train[indices2]

print(f"Sampled daily Training data shape: {X_lying_sampled.shape}")  
print(f"Sampled daily Target shape: {y_lying_sampled.shape}")

Sampled daily Training data shape: (736, 5, 4)
Sampled daily Target shape: (736, 3)


In [22]:
# 기존의 훈련데이터와 또다시 샘플링된 일상데이터 그리고 샘플링된 빗나간 훈련데이터를 합치는 과정 
final_X_combine=np.concatenate((X_combined,X_lying_sampled),axis=0) 
final_y_combine=np.concatenate((y_combined,y_lying_sampled),axis=0) 

print("Final Combined Training data shape:", final_X_combine.shape)  
print("Final Combined Target shape:", final_y_combine.shape)

Final Combined Training data shape: (2208, 5, 4)
Final Combined Target shape: (2208, 3)


# train dataset 생성완료

## train_test_split 진행

In [28]:
from sklearn.model_selection import train_test_split

# 클래스 라벨 추출
y_labels = np.argmax(final_y_combine, axis=1)

# train과 test set으로 분할. stratify 옵션으로 클래스 비율 유지
X_train, X_test, y_train, y_test = train_test_split(final_X_combine, final_y_combine,
                                                    stratify=y_labels,
                                                    test_size=0.2,
                                                    random_state=42)

print("Training data shape:", X_train.shape)
print("Training labels shape:", y_train.shape)
print("Testing data shape:", X_test.shape)
print("Testing labels shape:", y_test.shape)

Training data shape: (1766, 5, 4)
Training labels shape: (1766, 3)
Testing data shape: (442, 5, 4)
Testing labels shape: (442, 3)


In [29]:
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

model = keras.Sequential()
model.add(keras.layers.LSTM(units=128, input_shape=(5, 4), return_sequences=True))
model.add(keras.layers.BatchNormalization())  # Batch Normalization 추가
model.add(keras.layers.LSTM(units=128))
model.add(keras.layers.BatchNormalization())  # Batch Normalization 추가
model.add(keras.layers.Dense(units=64, activation='relu', kernel_regularizer=keras.regularizers.l2(0.01))) 
model.add(keras.layers.Dense(units=3, activation='softmax'))

In [30]:
# Learning Rate Scheduler 추가
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5)

# Model Checkpointing 추가
model_checkpoint = keras.callbacks.ModelCheckpoint(
    "latest_sequential_model.h5", save_best_only=True)

# Early Stopping
early_stopping = EarlyStopping(patience=10, restore_best_weights=True)

model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

history = model.fit(X_train, y_train,
                    validation_data=(X_test, y_test),
                    epochs=100,
                    batch_size=32,
                    callbacks=[early_stopping, lr_scheduler, model_checkpoint])

Epoch 1/100
56/56 [==============================] - 3s 15ms/step - loss: 1.5074 - accuracy: 0.6857 - val_loss: 1.6283 - val_accuracy: 0.6380 - lr: 0.0010
Epoch 2/100
56/56 [==============================] - 0s 5ms/step - loss: 1.2124 - accuracy: 0.7565 - val_loss: 1.3936 - val_accuracy: 0.7081 - lr: 0.0010
Epoch 3/100
56/56 [==============================] - 0s 5ms/step - loss: 1.0716 - accuracy: 0.7769 - val_loss: 1.2262 - val_accuracy: 0.6765 - lr: 0.0010
Epoch 4/100
56/56 [==============================] - 0s 5ms/step - loss: 0.9463 - accuracy: 0.7860 - val_loss: 1.1042 - val_accuracy: 0.7738 - lr: 0.0010
Epoch 5/100
56/56 [==============================] - 0s 6ms/step - loss: 0.8701 - accuracy: 0.7865 - val_loss: 0.9508 - val_accuracy: 0.7489 - lr: 0.0010
Epoch 6/100
56/56 [==============================] - 0s 6ms/step - loss: 0.8101 - accuracy: 0.7928 - val_loss: 0.9083 - val_accuracy: 0.7534 - lr: 0.0010
Epoch 7/100
56/56 [==============================] - 0s 5ms/step - loss: 0.

56/56 [==============================] - 0s 5ms/step - loss: 0.2462 - accuracy: 0.9298 - val_loss: 0.3916 - val_accuracy: 0.8665 - lr: 1.0000e-06
Epoch 54/100
56/56 [==============================] - 0s 5ms/step - loss: 0.2472 - accuracy: 0.9315 - val_loss: 0.3913 - val_accuracy: 0.8643 - lr: 1.0000e-06
Epoch 55/100
56/56 [==============================] - 0s 5ms/step - loss: 0.2423 - accuracy: 0.9309 - val_loss: 0.3913 - val_accuracy: 0.8688 - lr: 1.0000e-06
Epoch 56/100
56/56 [==============================] - 0s 5ms/step - loss: 0.2408 - accuracy: 0.9292 - val_loss: 0.3908 - val_accuracy: 0.8665 - lr: 1.0000e-06
Epoch 57/100
56/56 [==============================] - 0s 5ms/step - loss: 0.2363 - accuracy: 0.9320 - val_loss: 0.3908 - val_accuracy: 0.8688 - lr: 1.0000e-06
Epoch 58/100
56/56 [==============================] - 0s 5ms/step - loss: 0.2507 - accuracy: 0.9281 - val_loss: 0.3909 - val_accuracy: 0.8688 - lr: 1.0000e-07
